<a href="https://colab.research.google.com/github/CodeandoMexico/hacking-civico/blob/master/notebooks/10_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="http://codeandomexico.org/resources/img/codeandomexico.png" width="500" alt="Codeando México"><br>
<a href="http://www.codeandomexico.org/" target="_blank"><img src="https://img.shields.io/badge/website-CodeandoMexico-00D88E.svg"></a>
<a href="http://slack.codeandomexico.org/" target="_blank"><img src="https://img.shields.io/badge/slack-CodeandoMexico-EC0E4F.svg"></a>
</p>
<!-- __ -->

# Curso de Datos Abiertos y Hacking Cívico

Este curso tiene como objetivo habilitar las capacidades de la ciudadanía y los servidores públicos en el uso y generación de datos abiertos para el bien común. Puedes encontrar más información [aquí](https://github.com/CodeandoMexico/hacking-civico).

## Petición de info

Primero procederemos a escoger el sitio web del que extraeremos información. Para ello, realizaremos una petición de información con el paquete `requests` en Python.

Importamos el paquete:

In [ ]:
import requests

Mandamos la petición HTTP:

In [ ]:
url = "https://coronavirus.bcs.gob.mx/casos-covid-19/"
sitio = requests.get(url)

Y podemos observar que carga todo el contenido del sitio.

In [ ]:
sitio.content

Sin embargo, notaremos que todo el contenido ha sido descargado y se despliega como una línea (cadena) de texto. Haremos uso del paquete `bs4` para cargar la información en un formato legible.

## Creando una sopa de letras

Importaremos el paquete `bs4` para dar formato a la información extraída y con ello poder buscar elementos específicos.

In [ ]:
from bs4 import BeautifulSoup


sopa = BeautifulSoup(sitio.content, 'html.parser')

Imprimamos la información en un formato como aparece en el navegador.

In [ ]:
print(sopa.prettify())

Ahora procederemos a extraer los primeros elementos de información, los contadores.

Nos apoyaremos de dos funciones para buscar información:

- `find`
- `find_all`

In [ ]:
titulos_ = sopa.find_all('', class_='') # <div class="elementor-counter-title">
nums_ = sopa.find_all('', class_='') # <span class="elementor-counter-number">

In [ ]:
print(titulos_)
print(nums_)

Necesitamos procesar los elementos de la información extraída, para ello, bs4 cuenta con la función `get_text()`.

In [ ]:
titulos = ???
nums = ???

In [ ]:
print(titulos)
print(nums)

Notemos que hay espacioes intermedios, así que podemos hacer un poco de procesamiento con `re`.

In [ ]:
import re


titulos = [re.sub(' ', '', titulo) for titulo in titulos]

In [ ]:
print(titulos)
print(nums)

Podemos crear una lista de tuplas:

In [ ]:
list(zip(titulos, nums))

**¡Felicidades!**

Acabas de dar tus primeros pasos extrayendo un conjunto muy pequeño de datos.

Pero piensa lo siguiente: este simple proceso puede ser empaquetado y podrías desplegar un scraper en la nube que esté conectado con un bot para estarte dando los números de manera automatizada, digamos, vía SMS.


**Ahora procederemos a la extracción de elementos más complejos, como una tabla.**

In [ ]:
tabla = sopa.find('', class_='') # <table class="jet-table">
tabla

De la tabla que hemos encontrado, podemos extraer dos elementos principales:

- La cabecera - Contiene los nombres de las columnas
- El cuerpo - Contiene los datos/renglones a extraer

Del código extraemos el renglón de cabecera:

In [ ]:
t_cabecera = tabla.find('') # <thead>
t_cabecera

In [ ]:
cols = t_cabecera.find_all('', class_='') # <div class="jet-table__cell-text">
col_nombres = [elem.get_text() for elem in cols]
col_nombres

Hacemos exactamente lo mismo con el cuerpo:

In [ ]:
t_cuerpo = tabla.find('') # <tbody>
t_cuerpo

In [ ]:
filas = t_cuerpo.find_all('', class_='') # <div class="jet-table__cell-text">
datos = [elem.get_text() for elem in filas]
datos

In [ ]:
datos, col_nombres

Nos damos cuenta de que los renglones no se encuentran en el mejor formato, así que podemos proceder a darles una mejor estructura.

Como la cantidad de elementos por renglon depende de la longitud de columnas, utilizaremos esa info para construir una tabla:

In [ ]:
n_cols = len(col_nombres)
n_rengs = len(datos) // n_cols

print(n_cols, n_rengs)

In [ ]:
t_ = []

for i in range(n_rengs):
    t_.append(datos[i * 4:i * 4 + 4])

Verificamos:

In [ ]:
t_

Y con los datos extraídos, podemos crear un DataFrame utilizando Pandas, como ya conocemos.

In [ ]:
import pandas as pd


tabla_datos = pd.DataFrame(t_, columns=col_nombres)
tabla_datos